In [1]:
import sys

import MDAnalysis as mda
from MDAnalysis.analysis import dihedrals
from MDAnalysis.analysis import contacts
from statistics import mean, stdev
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline
import time
import glob
import math

sys.path.append('/home/khlee/repositories/pia-x/coupler/scripts')
from _heatmap import *


In [17]:
# testcase
#PDB="infiles/d3_gi_pd_mod.pdb"
#DCDs=glob.glob('infiles/test_d3_gi_pd_*.dcd')
#DCDs=glob.glob('infiles/test_d3_gi_pd_4.dcd')


PDB="/home/khlee/desmond/output/drd3/pd2/drd3_pd2.f2/init_ref.protein.pdb"
DCDs=glob.glob('/home/khlee/desmond/output/drd3/pd2/drd3_pd2.f2/drd3_pd2.f2_e0-2400.protein.wrapped.dcd')
u = mda.Universe(PDB, DCDs)


In [3]:
# define
# receptor
rec_chainid='R'
rec_segments=['27:57', '63:91', '100:134', '145:170', '186:218', '322:354', '362:400']
rec_labels=['TM1','TM2','TM3','TM4','TM5','TM6','TM7']
# Ga
ga_chainid='A'
ga_segments=['7:31','47:56','242:255','271:280', '296:311','329:351']
# need to refine above definition

# steal from https://www.uniprot.org/uniprot/P63096


In [4]:

# atomselection
segR_CA = u.select_atoms('protein and segid C'+rec_chainid+' and name CA and resid 27:57 or resid 63:91 or resid 100:134 or resid 145:170 or resid 186:218 or resid 322:354 or resid 362:400')
segR_I = segR_CA.moment_of_inertia()
# principal_axes
segR_UT = segR_CA.principal_axes()

# atomselection
segA_CA = u.select_atoms('protein and segid C'+ga_chainid+' and name CA')
segA_I = segA_CA.moment_of_inertia()
# principal_axes
segA_UT = segA_CA.principal_axes()


In [5]:
segA_UT

array([[ 0.78168562, -0.03601454, -0.62263195],
       [ 0.49370697, -0.57427138,  0.6530435 ],
       [ 0.38107877,  0.81787245,  0.43111905]])

In [6]:
from pylab import cross,dot,inv
# https://stackoverflow.com/questions/36409140/create-a-rotation-matrix-from-2-normals
from scipy.spatial.transform import Rotation as R
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html

# given two vectors and calculate rotation matrix
def rot(U,V):
    W=cross(U,V)
    A=np.array([U,W,cross(U,W)]).T
    B=np.array([V,W,cross(V,W)]).T
    return dot(B,inv(A))

# unittest
# put this in unittest module
Va = np.array([ 1, 0, 100])
Vb = np.array([ 0, 1, 0.5])
rotM = rot(Va,Vb)
assert [Vb[i]==rotM.dot(Va)[i] for i in range(len(Va))]

In [7]:
# Va for segR
# Vb for segA
Va=segR_UT[0]+segR_UT[1]+segR_UT[2]
Vb=segA_UT[0]+segA_UT[1]+segA_UT[2]
rotM = rot(Va,Vb)
rotM2=R.from_matrix(rotM)
rotM2.as_euler('zyx', degrees=True)
#need to confirm this is right

array([-62.70551459, -41.27573391,  -1.47613562])

In [8]:
# Va for segR
# Vb for segA
Va=segR_UT[0]+segR_UT[1]+segR_UT[2]
Vb=segA_UT[0]+segA_UT[1]+segA_UT[2]
rotM = rot(Va,Vb)
rotM2=R.from_matrix(rotM)
rotM2.as_euler('xyx', degrees=True)
#need to confirm this is right

array([ 134.64677703,   69.84073661, -110.2751003 ])

In [ ]:
PDB="/home/khlee/desmond/output/drd3/pd2/drd3_pd2.f2/init_ref.protein.pdb"
DCDs=glob.glob('/home/khlee/desmond/output/drd3/pd2/drd3_pd2.f2/drd3_pd2.f2_e0-2400.protein.wrapped.dcd')

u = mda.Universe(PDB, DCDs)

In [18]:
for ts in u.trajectory:
    #print (ts)

    # atomselection
    segR_CA = u.select_atoms('protein and segid C'+rec_chainid+' and name CA and resid 27:57 or resid 63:91 or resid 100:134 or resid 145:170 or resid 186:218 or resid 322:354 or resid 362:400')
    segR_I = segR_CA.moment_of_inertia()
    # principal_axes
    segR_UT = segR_CA.principal_axes()

    # atomselection
    segA_CA = u.select_atoms('protein and segid C'+ga_chainid+' and name CA')
    segA_I = segA_CA.moment_of_inertia()
    # principal_axes
    segA_UT = segA_CA.principal_axes()

    Va=segR_UT[0]+segR_UT[1]+segR_UT[2]
    Vb=segA_UT[0]+segA_UT[1]+segA_UT[2]
    rotM = rot(Va,Vb)
    rotM2=R.from_matrix(rotM)
    rotM2.as_euler('zyx', degrees=True)
    print(rotM2.as_euler('xyx', degrees=True))
    


[ 164.7849741    33.4349781  -114.69861089]
[ 164.57353715   33.57597927 -114.22199538]
[ 164.51426508   33.73355831 -113.8803517 ]
[ 164.8526133    33.50299766 -114.65164904]
[ 164.83881544   33.46117407 -114.71058011]
[ 163.98511199   33.42882211 -113.85087743]
[ 163.64802857   33.74543809 -112.93780198]
[ 164.405455     33.8062729  -113.63554982]
[ 164.78780478   33.70340902 -114.22756636]
[ 164.81396336   33.93233507 -113.85057105]
[ 163.71503296   33.87319509 -112.78431261]
[ 163.12386628   33.64153661 -112.568166  ]
[ 163.46604767   32.94715783 -114.1402198 ]
[ 163.17408658   33.11486128 -113.53896981]
[ 163.7192894    33.25231589 -113.87694234]
[ 164.67325853   33.48726919 -114.48579605]
[ 163.40903498   33.35134186 -113.37495633]
[ 164.18156431   33.87172162 -113.28134218]
[ 162.53464766   33.32818527 -112.49770967]
[ 162.67454682   33.28929212 -112.71115594]
[ 163.71674462   33.38444898 -113.64307223]
[ 162.86137561   33.31639595 -112.85954384]
[ 162.31307477   33.30946575 -11

[ 165.04547605   33.73943874 -114.44195321]
[ 164.67911785   33.90265545 -113.75831191]
[ 164.84686951   34.02496135 -113.72176831]
[ 163.63792615   33.38198585 -113.56373567]
[ 162.97947712   32.80734382 -113.86763371]
[ 163.67140835   33.39795743 -113.57130872]
[ 162.02510551   33.05635272 -112.43831534]
[ 163.05700046   33.30820594 -113.0790959 ]
[ 164.32297642   34.05337527 -113.11128276]
[ 163.99576866   33.76229774 -113.2764756 ]
[ 164.02308447   33.25885886 -114.18943219]
[ 162.93081334   32.96750378 -113.5384861 ]
[ 163.45357698   32.9220197  -114.17074011]
[ 164.39307676   33.35260848 -114.42139727]
[ 163.85030193   33.43578557 -113.69510339]
[ 163.47170968   33.28620684 -113.55498188]
[ 163.2187449    33.8256601  -112.34581448]
[ 163.28117069   33.17017178 -113.55578523]
[ 163.48462714   33.34899376 -113.45900177]
[ 163.78367319   33.31590502 -113.8341879 ]
[ 163.0070293    33.15990947 -113.28453631]
[ 162.10515257   33.47819894 -111.79236049]
[ 162.63509512   33.83910892 -11

[ 162.02559531   32.30095105 -113.73854722]
[ 161.49117373   32.68491706 -112.52454407]
[ 162.69053035   32.23512031 -114.55139741]
[ 161.79875198   32.50091017 -113.15879347]
[ 161.2893113    32.16094455 -113.2117066 ]
[ 161.67668889   31.82343705 -114.19083823]
[ 159.81651118   31.77080956 -112.36151539]
[ 160.97456878   32.52552188 -112.26538919]
[ 162.76217228   32.34748352 -114.43341102]
[ 162.38509504   31.94872432 -114.72108964]
[ 160.82309027   32.29914178 -112.49559267]
[ 162.30981016   32.67084898 -113.39962268]
[ 161.20999081   32.09250132 -113.24626626]
[ 161.5847965    31.97765105 -113.83159779]
[ 161.5972444    32.61899453 -112.7471993 ]
[ 162.61695085   32.57303451 -113.89077496]
[ 161.90675444   32.22165704 -113.75028385]
[ 162.43200267   32.98802367 -112.97972622]
[ 161.78177969   32.93367888 -112.3976835 ]
[ 161.18640735   32.49471714 -112.53571066]
[ 162.67167913   32.85069366 -113.46834221]
[ 162.34324038   32.94743551 -112.95721214]
[ 162.82898953   32.86198402 -11

[ 162.04002328   33.57870842 -111.55126387]
[ 162.81795306   33.78055178 -112.00613172]
[ 163.49779023   34.02523384 -112.28867758]
[ 162.87660801   33.3278755  -112.85554456]
[ 161.51832255   32.94299534 -112.10982477]
[ 162.43718648   33.64550595 -111.84572475]
[ 163.573203     33.6425873  -113.03905909]
[ 164.03717399   34.08518952 -112.75176767]
[ 163.19254945   34.4021017  -111.3083059 ]
[ 163.23397841   33.85472259 -112.31092316]
[ 163.10581721   33.57437929 -112.66645132]
[ 163.68107263   33.83434581 -112.8166829 ]
[ 162.33587241   33.34769968 -112.25715498]
[ 162.35562383   33.48302621 -112.04313784]
[ 163.00823284   33.3491102  -112.95665375]
[ 164.7354922    33.40072282 -114.70556992]
[ 162.26712835   33.4590799  -111.99294574]
[ 163.19105477   33.62005471 -112.67614385]
[ 163.27336565   33.55075916 -112.8836579 ]
[ 163.09480238   33.46907804 -112.83854485]
[ 163.38867861   33.44206249 -113.19502229]
[ 163.80220099   33.8043494  -112.99736391]
[ 164.56691783   34.02802873 -11

[ 167.85076764   35.34580645 -114.62929562]
[ 167.39011916   35.4408153  -113.9438531 ]
[ 166.570717     35.02633865 -113.79669816]
[ 166.4793496    34.80641691 -114.09428358]
[ 167.77826552   34.89480388 -115.37274422]
[ 167.43627413   35.4185927  -114.03551185]
[ 166.77251687   35.34823563 -113.43354103]
[ 169.07423291   35.96224989 -114.86739723]
[ 169.3581718    35.69548543 -115.68650103]
[ 168.79154947   35.78646007 -114.87375169]
[ 167.66910317   35.19076667 -114.71043895]
[ 167.83171939   35.22485499 -114.8295234 ]
[ 167.79888407   35.1346359  -114.95786137]
[ 165.76787749   34.53505563 -113.80681049]
[ 167.65937697   35.26612188 -114.56185817]
[ 166.37373442   34.32642273 -114.84303188]
[ 166.00655666   34.45754741 -114.20546903]
[ 165.60038354   34.98680348 -112.81520292]
[ 166.67659791   34.49505071 -114.87302821]
[ 168.3996224    35.11456612 -115.66982071]
[ 167.68706522   34.81253289 -115.42056621]
[ 165.57964848   34.54067066 -113.59285369]
[ 166.36198031   34.65621003 -11

...

[ 169.88514115   36.01749995 -115.69036535]
[ 169.31477993   36.15250641 -114.78598006]
[ 170.81747574   36.43942975 -115.97628083]
[ 169.5946899    36.08335787 -115.23411278]
[ 167.24264799   35.50812866 -113.65815997]
[ 168.10240499   35.26509135 -115.05887074]
[ 166.93979812   35.45315119 -113.42576372]
[ 168.60121944   35.83485964 -114.56954817]
[ 169.46957377   36.37365544 -114.54851586]
[ 169.28476396   36.1781936  -114.7039573 ]
[ 168.01792513   35.42003156 -114.6798964 ]
[ 167.71367817   35.69986663 -113.8273468 ]
[ 169.2943691    36.25316789 -114.57488845]
[ 168.79292686   36.25815145 -113.99955232]
[ 169.56709785   35.85455177 -115.62955341]
[ 168.05935671   35.81736656 -113.99525702]
[ 167.09162215   35.18002059 -114.08995125]
[ 168.60400511   35.78533467 -114.66432154]
[ 169.56644259   36.16626461 -115.0468774 ]
[ 168.40464725   35.63402101 -114.71961908]
[ 167.41723158   35.18741898 -114.43655348]
[ 167.97565135   35.40423546 -114.66162499]
[ 167.51480015   35.24774482 -11

[ 167.82057963   35.71784872 -113.91289248]
[ 169.61429773   36.36608495 -114.7274152 ]
[ 170.75262842   36.67252347 -115.45942177]
[ 171.37665281   37.05486664 -115.45907132]
[ 169.76941027   36.80475423 -114.07964823]
[ 169.52700751   36.4631534  -114.44612051]
[ 170.1329334    36.88465998 -114.34345151]
[ 170.12255803   36.11643564 -115.77860213]
[ 168.10735523   35.67881372 -114.3040381 ]
[ 169.35719681   35.94313571 -115.22461599]
[ 168.55935216   36.08044249 -114.06756769]
[ 169.47365327   35.74746653 -115.72196157]
[ 170.04333945   36.39939608 -115.15584816]
[ 170.00354922   36.75382175 -114.44262485]
[ 168.62737747   36.24835166 -113.83208622]
[ 168.20108929   36.10448788 -113.62290797]
[ 167.58155448   35.89414589 -113.32437749]
[ 168.76927612   35.90062111 -114.63703214]
[ 168.98985342   36.48897507 -113.79058278]
[ 168.68812957   36.52657279 -113.38225371]
[ 169.98294096   36.8258087  -114.28322381]
[ 168.15150962   36.20681965 -113.37844672]
[ 169.44015104   36.42087935 -11

[ 169.30187965   36.44796119 -114.21924478]
[ 168.85971809   36.29176555 -114.0120428 ]
[ 170.74595337   36.78521319 -115.23794388]
[ 169.2075485    36.49779751 -114.01936775]
[ 167.72122548   36.25304595 -112.81733364]
[ 168.7283562    36.3790308  -113.70211744]
[ 168.97699691   36.21513017 -114.28684276]
[ 170.14293284   36.62301243 -114.84906945]
[ 168.68631311   36.25519618 -113.88540301]
[ 170.16987505   36.55382223 -115.01049683]
[ 170.20745971   37.14191868 -113.94139385]
[ 169.33835148   36.87244117 -113.46439512]
[ 169.1491816    36.90217872 -113.19587432]
[ 169.89178846   37.12783798 -113.60862946]
[ 170.17563172   36.8018906  -114.54883189]
[ 170.69641554   37.15688549 -114.47409485]
[ 168.83583259   36.68928287 -113.2436673 ]
[ 168.3279028    35.77154858 -114.37973951]
[ 169.23848877   36.36041953 -114.31120918]
[ 169.49784131   36.31767821 -114.68544668]
[ 167.84228071   35.68050855 -114.00562724]
[ 168.58814511   36.24325532 -113.7976321 ]
[ 168.25301707   35.75237518 -11

[ 169.49226827   36.38612064 -114.55097092]
[ 169.32987628   36.05708025 -114.98114422]
[ 167.2118058    35.39634794 -113.82826361]
[ 167.97524084   35.58207712 -114.33455739]
[ 167.38697639   35.56897581 -113.70600594]
[ 168.48235184   36.20946308 -113.74206429]
[ 168.82209541   35.91639344 -114.66737258]
[ 167.7601079    35.81326735 -113.67040365]
[ 167.85917677   35.84707241 -113.71802476]
[ 168.31618507   35.91081932 -114.10938433]
[ 168.06359627   35.89462397 -113.85753517]
[ 169.63309785   36.49505731 -114.50695438]
[ 168.4303335    36.16024966 -113.77523033]
[ 168.07857177   36.18478655 -113.3382671 ]
[ 170.29436118   36.78844063 -114.71053306]
[ 170.76381422   37.05781445 -114.74053447]
[ 170.17447874   36.65324176 -114.82825611]
[ 168.13328798   35.99345092 -113.75269249]
[ 169.13675923   36.39662399 -114.12858459]
[ 169.44957766   36.75637368 -113.80803151]
[ 168.26250605   36.37874133 -113.18311026]
[ 168.62772201   36.65966516 -113.06669456]
[ 168.75744391   36.31181199 -11

[ 167.75837377   35.72877802 -113.82378662]
[ 167.88834405   35.77708508 -113.8792082 ]
[ 171.04305437   36.78541639 -115.58307732]
[ 170.09471814   36.43753712 -115.14313809]
[ 168.73068993   35.99001193 -114.42776166]
[ 168.00672389   36.28805449 -113.06766371]
[ 168.74122756   36.18588018 -114.07588498]
[ 168.10160072   36.20562112 -113.32527283]
[ 168.32176397   36.15951451 -113.65544974]
[ 169.10331287   36.01936915 -114.79411   ]
[ 168.26161687   36.34696931 -113.24105997]
[ 168.97936584   36.81096488 -113.1765958 ]
[ 170.22989982   36.69271965 -114.8173843 ]
[ 170.39895442   36.7744051  -114.8574205 ]
[ 169.2363528    36.78168474 -113.51996406]
[ 170.43548879   36.87277883 -114.71317946]
[ 168.78035191   36.57543328 -113.39422844]
[ 169.94285533   36.55290803 -114.75181019]
[ 168.94052004   36.52870399 -113.66095756]
[ 168.33630467   36.00113214 -113.96487514]
[ 167.8779496    36.10298891 -113.2671624 ]
[ 168.68814104   36.43401934 -113.55470337]
[ 168.77962277   36.27531836 -11

In [22]:
PDB="/home/khlee/desmond/output/drd3/pd2/drd3_pd2.f1/init_ref.protein.pdb"
DCDs=glob.glob('/home/khlee/desmond/output/drd3/pd2/drd3_pd2.f1/drd3_pd2.f1_e0-2400.protein.wrapped.dcd')

u = mda.Universe(PDB, DCDs)

In [23]:
for ts in u.trajectory:
    #print (ts)

    # atomselection
    segR_CA = u.select_atoms('protein and segid C'+rec_chainid+' and name CA and resid 27:57 or resid 63:91 or resid 100:134 or resid 145:170 or resid 186:218 or resid 322:354 or resid 362:400')
    segR_I = segR_CA.moment_of_inertia()
    # principal_axes
    segR_UT = segR_CA.principal_axes()

    # atomselection
    segA_CA = u.select_atoms('protein and segid C'+ga_chainid+' and name CA')
    segA_I = segA_CA.moment_of_inertia()
    # principal_axes
    segA_UT = segA_CA.principal_axes()

    Va=segR_UT[0]+segR_UT[1]+segR_UT[2]
    Vb=segA_UT[0]+segA_UT[1]+segA_UT[2]
    rotM = rot(Va,Vb)
    rotM2=R.from_matrix(rotM)
    rotM2.as_euler('zyx', degrees=True)
    print(rotM2.as_euler('xyx', degrees=True))
    


[ 158.94346495   33.27680148 -108.95129557]
[ 159.86645114   33.2083446  -109.98162423]
[ 159.8189848    34.04869059 -108.50541297]
[ 161.31080026   33.49216152 -110.95358901]
[ 160.26163783   33.75764982 -109.44089462]
[ 160.3906015    33.62794588 -109.79133645]
[ 159.26290974   33.22235632 -109.35809795]
[ 159.34852159   33.91408413 -108.27166952]
[ 161.05277126   33.47359965 -110.72313678]
[ 159.22042322   33.10956442 -109.50649079]
[ 158.52975222   33.02290193 -108.97204942]
[ 158.62617137   33.06326917 -108.99885474]
[ 158.72459021   33.68471466 -108.0498175 ]
[ 159.93806757   33.84291434 -108.9737391 ]
[ 161.07920998   33.85995232 -110.08607878]
[ 160.89381484   33.50894633 -110.50147253]
[ 160.10710307   32.89565446 -110.75377897]
[ 160.84925129   33.31483585 -110.78886751]
[ 160.7354348    33.23242596 -110.81453579]
[ 159.11648252   33.18939678 -109.26907071]
[ 159.58555769   33.24460414 -109.64030628]
[ 158.59363616   32.89671624 -109.24638318]
[ 159.96234047   33.20477385 -11

[ 159.45717121   34.1346214  -108.00373192]
[ 160.84053583   34.0381231  -109.53951459]
[ 159.45547147   34.35654473 -107.62478   ]
[ 159.95479833   34.04063871 -108.65321827]
[ 159.83763006   33.40346361 -109.62157814]
[ 161.06944366   34.10846308 -109.6484593 ]
[ 159.84036683   34.0402296  -108.54091903]
[ 159.45798755   34.01465514 -108.20832457]
[ 161.06235002   34.86102895 -108.34245031]
[ 161.12775123   34.44834706 -109.12086969]
[ 160.73925784   34.57560851 -108.51376783]
[ 160.44095527   34.45584263 -108.42375779]
[ 159.80320462   33.88895969 -108.76186768]
[ 160.47255423   34.1036405  -109.05922928]
[ 161.18704212   34.48653298 -109.11434615]
[ 161.8618472    33.83373987 -110.92692545]
[ 160.41231473   33.90862894 -109.3331334 ]
[ 159.98477943   34.12186593 -108.54424155]
[ 160.05213826   33.60600335 -109.49068843]
[ 160.40323634   33.99068843 -109.18363411]
[ 160.83309325   34.5662298  -108.6232429 ]
[ 159.67618334   34.40819841 -107.75244885]
[ 160.33765844   34.39638431 -10

[ 158.96571723   33.73375978 -108.20224074]
[ 158.5970474    33.54636646 -108.15854708]
[ 158.37769265   33.6126877  -107.83398184]
[ 159.29197445   33.72593268 -108.5350431 ]
[ 157.94019724   33.2017134  -108.09841705]
[ 158.17193917   33.24951483 -108.24324257]
[ 158.05728009   33.42229287 -107.84295802]
[ 158.51916927   33.80035491 -107.6555811 ]
[ 157.93224137   33.76521139 -107.14854667]
[ 158.27520162   33.95370216 -107.16212067]
[ 160.3902564    34.31012155 -108.62349497]
[ 160.03339767   34.45860997 -108.01677754]
[ 160.0104646    34.10399133 -108.60013169]
[ 158.7127554    33.99827478 -107.50963693]
[ 159.26903635   33.65710523 -108.62903865]
[ 159.00107183   33.01045862 -109.45667005]
[ 158.86020651   33.52585182 -108.44991073]
[ 159.69303347   33.65064571 -109.05816337]
[ 159.70240681   33.48183659 -109.35409676]
[ 159.4689568    33.68154101 -108.78440662]
[ 160.5477508    33.69586649 -109.83272181]
[ 160.71726692   34.23955552 -109.06998108]
[ 160.10887379   34.25867126 -10

[ 162.19959055   34.73315926 -109.70828809]
[ 160.90751307   35.0085708  -107.93369681]
[ 162.9185685    34.87273046 -110.19862898]
[ 161.49944182   34.51655616 -109.37637201]
[ 161.86131071   34.42185794 -109.90662226]
[ 162.96303662   34.82783894 -110.32322569]
[ 162.93338922   34.45699305 -110.94345938]
[ 164.61396657   34.86579248 -111.97899821]
[ 164.38337678   34.56595174 -112.26791545]
[ 161.92697336   34.71907553 -109.45633142]
[ 163.4297063    34.50069125 -111.38174361]
[ 162.71810048   34.90484604 -109.93670158]
[ 162.61720283   34.67977912 -110.22781944]
[ 161.99737816   34.21964791 -110.39632559]
[ 162.85626739   34.37097217 -111.01442941]
[ 162.44291463   34.44862201 -110.45327932]
[ 161.33361676   34.3825454  -109.44132127]
[ 160.96114994   34.08803832 -109.57475715]
[ 160.97620078   34.56958399 -108.76007327]
[ 160.47667704   34.68029514 -108.0735351 ]
[ 162.08818381   34.5751154  -109.87055774]
[ 161.66797148   34.11795864 -110.23722489]
[ 162.92279386   34.89203281 -11

[ 162.15742665   34.76064425 -109.61749153]
[ 160.8727746    34.29700089 -109.12654175]
[ 160.96173128   33.89032759 -109.9153649 ]
[ 160.82833533   34.10994954 -109.40382866]
[ 161.80337211   33.93557023 -110.69087994]
[ 160.97812208   34.1523353  -109.48113516]
[ 161.54329698   34.29640044 -109.8019271 ]
[ 162.4177929    34.50215277 -110.33409802]
[ 161.89629923   34.35881305 -110.05166893]
[ 161.64997583   34.0282002  -110.3742586 ]
[ 161.1181086    34.05323162 -109.79259773]
[ 161.51718176   34.30641759 -109.75818116]
[ 160.9765735    33.81166652 -110.06551252]
[ 160.2798362    33.90838497 -109.20149855]
[ 162.13629293   34.15471999 -110.65111585]
[ 161.6844793    34.13622554 -110.22237377]
[ 158.88746452   33.18099957 -109.05758499]
[ 159.32029391   34.21701522 -107.73003402]
[ 161.50805975   34.25566682 -109.83678265]
[ 161.2027941    34.4608045  -109.17459686]
[ 162.57411113   34.66423396 -110.21088044]
[ 161.20183226   34.34337171 -109.37646142]
[ 162.02458763   34.69264253 -10

...

[ 161.99371097   33.75804691 -111.1931554 ]
[ 160.94442059   33.85379444 -109.96070997]
[ 160.6938099    33.65311371 -110.05265658]
[ 159.98472711   33.71334215 -109.2407793 ]
[ 159.70714022   33.0006405  -110.17461088]
[ 160.44903969   33.03037556 -110.86960262]
[ 159.8644615    33.65466413 -109.22123617]
[ 159.88173183   33.40894678 -109.65618376]
[ 160.26742039   33.76263947 -109.43814182]
[ 161.23183174   33.64277797 -110.61420892]
[ 160.77641437   33.60372402 -110.22047147]
[ 160.80344983   34.00132015 -109.56554313]
[ 161.20022059   33.81209038 -110.29079649]
[ 161.11937027   33.95535231 -109.96243609]
[ 162.06863121   34.42035794 -110.12006799]
[ 161.21999539   33.77514943 -110.37441713]
[ 161.06513523   33.16310875 -111.26809511]
[ 160.43158536   33.70492716 -109.7008121 ]
[ 161.51071987   33.57518691 -111.01454641]
[ 160.87124246   33.76214849 -110.04436169]
[ 161.49460174   33.61940855 -110.92188066]
[ 162.85801209   33.89598957 -111.8464555 ]
[ 162.11509096   34.04815057 -11

[ 160.97333596   33.32603842 -110.89555234]
[ 161.87801339   33.18468859 -112.06482413]
[ 161.08347926   33.57408981 -110.58175985]
[ 162.54103606   33.54633951 -112.12570881]
[ 161.71986236   34.10751449 -110.30806371]
[ 160.19189773   33.36376932 -110.04291949]
[ 160.9448521    33.4833613  -110.59699396]
[ 159.9686368    32.90447832 -110.5995436 ]
[ 159.37761192   32.58034521 -110.55531501]
[ 159.82010633   32.4585127  -111.205096  ]
[ 159.00073422   32.1555248  -110.89278556]
[ 158.82421403   32.4935684  -110.15018772]
[ 161.53073526   33.25387851 -111.5882815 ]
[ 160.42943803   33.22715452 -110.51442368]
[ 160.74013729   32.82251836 -111.5192239 ]
[ 159.11733632   32.67648008 -110.13399978]
[ 160.94611687   33.12433483 -111.21328162]
[ 162.3679233    33.32047975 -112.33759144]
[ 161.39143342   33.36897502 -111.24770589]
[ 161.78073769   33.3049909  -111.75711867]
[ 160.98841772   33.77316798 -110.1436131 ]
[ 162.65371643   33.86675509 -111.68529421]
[ 162.09211097   33.52138001 -11

[ 162.42818164   34.33145837 -110.64259944]
[ 161.06334985   34.4001789  -109.13947578]
[ 162.25557484   34.43592539 -110.2837008 ]
[ 162.71303819   34.41234641 -110.79424514]
[ 161.97998344   34.01469391 -110.73427509]
[ 161.17515345   34.22457747 -109.55465832]
[ 162.13048191   34.15923421 -110.63732131]
[ 162.43912841   34.25529335 -110.7866554 ]
[ 162.90982169   34.15625258 -111.44557889]
[ 162.4321618    34.20467459 -110.86772787]
[ 161.02475758   34.00400481 -109.78332811]
[ 162.37334806   34.35939476 -110.53764034]
[ 161.16642374   33.87465719 -110.14889201]
[ 161.70114322   33.76452932 -110.8822904 ]
[ 162.1616384    34.29444005 -110.43406955]
[ 162.89005953   34.56316305 -110.71261781]
[ 162.92239828   34.47569819 -110.89931849]
[ 161.01925405   34.45495736 -109.00084034]
[ 161.98971704   34.26331234 -110.3126666 ]
[ 163.01123805   34.56080647 -110.84185402]
[ 161.58198503   33.90802774 -110.51284708]
[ 162.51569585   34.48524135 -110.46395707]
[ 162.67447716   33.96458195 -11

[ 163.73039787   34.48150555 -111.72969819]
[ 164.76445833   34.72959198 -112.38129268]
[ 162.27302023   34.34129635 -110.46642169]
[ 162.11565586   34.07955441 -110.76058522]
[ 163.33344593   34.53262725 -111.22536175]
[ 163.34863767   34.6449968  -111.04338816]
[ 163.15196244   34.87461661 -110.43557382]
[ 162.62698172   34.58935927 -110.39609629]
[ 162.43127477   34.32517426 -110.65673439]
[ 162.12306514   34.36871464 -110.26543201]
[ 163.6312782    34.53116395 -111.53840069]
[ 162.43850611   34.56212725 -110.25056704]
[ 165.21637355   35.33611075 -111.7770084 ]
[ 163.97189583   35.1830256  -110.74033332]
[ 164.51863556   35.24618064 -111.20020011]
[ 164.10880101   34.93436265 -111.32491772]
[ 163.89683425   35.01310237 -110.96358762]
[ 162.34327416   34.66252048 -109.97792454]
[ 162.67749513   34.41049918 -110.76086756]
[ 163.20096721   34.35940503 -111.3920066 ]
[ 162.78679148   34.19925772 -111.24294117]
[ 163.31552693   34.57714289 -111.12842528]
[ 163.47211424   34.91754756 -11

[ 163.66869878   35.41710912 -110.01020691]
[ 162.93743609   35.04543949 -109.91434123]
[ 164.90849735   35.66964634 -110.85225199]
[ 164.40093779   35.96708528 -109.78797068]
[ 163.68112853   35.59402001 -109.70893873]
[ 161.70705989   35.36406994 -108.11136405]
[ 163.07923741   35.19671381 -109.79331663]
[ 162.38680576   35.28713035 -108.92906343]
[ 164.01752792   35.65180435 -109.95371789]
[ 163.8892533    35.54694571 -110.00760007]
[ 163.34490575   35.63474962 -109.2913158 ]
[ 164.81312889   35.58545086 -110.90298233]
[ 166.65725473   35.83892658 -112.41452552]
[ 165.23516192   35.58413587 -111.35108215]
[ 163.75178915   35.26206509 -110.37125201]
[ 164.48651259   35.50505494 -110.704117  ]
[ 164.002729     35.23347677 -110.68283425]
[ 163.47136597   35.75078543 -109.21502229]
[ 164.26472402   35.52913561 -110.42938873]
[ 162.30679706   34.81878483 -109.66777532]
[ 163.10353104   35.01648216 -110.13587184]
[ 162.93308829   35.45786722 -109.18341185]
[ 162.58766045   34.89997485 -10

In [20]:
PDB="/home/khlee/desmond/output/drd3ao/pd2/drd3ao_pd2.f2/init_ref.protein.pdb"
DCDs=glob.glob('/home/khlee/desmond/output/drd3ao/pd2/drd3ao_pd2.f2/drd3ao_pd2.f2_e0-2400.protein.wrapped.dcd')
u = mda.Universe(PDB, DCDs)

In [21]:
for ts in u.trajectory:
    #print (ts)

    # atomselection
    segR_CA = u.select_atoms('protein and segid C'+rec_chainid+' and name CA and resid 27:57 or resid 63:91 or resid 100:134 or resid 145:170 or resid 186:218 or resid 322:354 or resid 362:400')
    segR_I = segR_CA.moment_of_inertia()
    # principal_axes
    segR_UT = segR_CA.principal_axes()

    # atomselection
    segA_CA = u.select_atoms('protein and segid C'+ga_chainid+' and name CA')
    segA_I = segA_CA.moment_of_inertia()
    # principal_axes
    segA_UT = segA_CA.principal_axes()

    Va=segR_UT[0]+segR_UT[1]+segR_UT[2]
    Vb=segA_UT[0]+segA_UT[1]+segA_UT[2]
    rotM = rot(Va,Vb)
    rotM2=R.from_matrix(rotM)
    rotM2.as_euler('zyx', degrees=True)
    print(rotM2.as_euler('xyx', degrees=True))
    


[148.92847105  33.7659691  -99.09008445]
[148.74689098  33.65199344 -99.11320282]
[149.34793079  33.74215705 -99.47788487]
[149.52848034  33.6935274  -99.70480837]
[149.60336359  34.04646065 -99.22291881]
[148.8412003   33.54070515 -99.36241135]
[148.83809575  33.86771773 -98.85884311]
[148.56821367  33.6640951  -98.94588394]
[147.81253445  33.47206243 -98.61190445]
[149.51538172  33.77576546 -99.56688118]
[150.52078061  34.23479181 -99.70373497]
[150.87782736  34.49598753 -99.59809381]
[150.2302268   33.96957926 -99.87049098]
[151.01229427  34.8382706  -99.17543017]
[150.17442014  34.30595565 -99.30053727]
[150.10277388  34.04280134 -99.64888152]
[149.84181495  33.70087356 -99.95824296]
[ 151.3450736    34.20143732 -100.45886795]
[ 151.64485402   34.043841   -100.9654327 ]
[149.82475188  34.49764212 -98.71031924]
[150.12112023  34.45358101 -99.02642553]
[149.88576363  34.46063321 -98.81855234]
[152.62382499  35.24803369 -99.89842457]
[ 152.39569333   34.92964167 -100.20975395]
[ 151.6

[148.89056362  33.30179795 -99.76993001]
[ 150.13596377   33.11922554 -101.11097117]
[ 149.72646919   33.56063189 -100.07747437]
[ 149.09731706   32.67187914 -100.91163337]
[ 150.89957438   33.03143363 -101.90868834]
[ 150.86300133   32.71211108 -102.37599679]
[ 151.41327451   33.01501965 -102.38370622]
[ 149.82045341   32.80556923 -101.32555305]
[ 151.03585452   32.64657358 -102.6301335 ]
[ 148.56992085   32.47507699 -100.7636162 ]
[ 150.34296467   32.18203706 -102.74643946]
[ 150.23004305   32.25022731 -102.54162686]
[ 149.08802828   32.64104314 -100.95105211]
[ 148.49549568   32.38520639 -100.83752182]
[ 149.78676174   32.5116655  -101.75102781]
[ 148.9861584    33.05944717 -100.22223856]
[ 149.43287834   32.5589977  -101.37279502]
[ 150.10407615   32.75649921 -101.6463395 ]
[ 150.07758281   32.72910276 -101.66591158]
[ 149.85599741   32.4047943  -101.97629037]
[ 148.52931832   32.87265129 -100.12204996]
[ 149.77942529   32.77904243 -101.33126471]
[148.68711105  33.06090613 -99.9674

[ 148.26603916   32.01890329 -101.20035391]
[ 148.2818257    32.07517183 -101.12814238]
[ 149.23855233   32.85253764 -100.75451479]
[ 150.05421406   32.91212095 -101.36190113]
[ 148.72254112   32.22903577 -101.26993583]
[ 148.19460206   32.64181792 -100.19155363]
[ 148.75379111   32.40583988 -101.02614851]
[ 147.49614971   31.70553244 -101.02007227]
[146.00448331  32.58844362 -98.45241069]
[ 148.00601767   32.57055379 -100.14096002]
[ 148.37697979   32.27452783 -100.90547824]
[ 149.45036016   32.01523314 -102.2256638 ]
[ 149.94941035   32.21017404 -102.35872666]
[ 148.24373317   32.55833883 -100.3600922 ]
[148.56522447  33.09177134 -99.81764857]
[146.64718306  32.28028032 -99.44228738]
[ 148.08091742   32.64520472 -100.09061057]
[ 147.90440959   32.38084837 -100.34326292]
[ 148.69587987   32.95966767 -100.12966687]
[ 149.15239109   32.49034732 -101.23759993]
[ 150.0717994    32.8599221  -101.45802346]
[ 149.11468588   33.09389631 -100.27821467]
[ 148.5395405    32.63508252 -100.4934559

[ 149.68294162   33.45750377 -100.19996274]
[ 149.79467377   33.4095715  -100.36904655]
[149.67111856  33.84886638 -99.58513811]
[ 149.19529427   33.21419533 -100.16161334]
[148.93898464  33.29604139 -99.81951599]
[148.76825653  33.49447912 -99.37215195]
[ 149.82173961   33.42975146 -100.36085535]
[148.74887123  33.48479335 -99.37075546]
[ 149.17359855   33.2202025  -100.1340142 ]
[149.89321674  33.77831359 -99.88184437]
[ 150.37908205   33.72384893 -100.37937848]
[ 150.47946688   33.80631133 -100.33661168]
[ 149.47802155   32.94369296 -100.81855927]
[ 150.44119004   32.97684849 -101.59571498]
[ 149.81029729   32.51698299 -101.76316447]
[ 149.80689086   32.45700655 -101.85296807]
[ 148.2791241    32.00417515 -101.23395515]
[ 149.13834932   32.51046491 -101.19465723]
[ 148.71261473   32.52381762 -100.81054004]
[ 149.24043979   31.58774148 -102.70079907]
[ 150.41873803   31.85364813 -103.3235807 ]
[ 149.39346339   32.33890466 -101.67787859]
[ 149.82041237   32.24444552 -102.19340521]
[ 1

[148.80741263  33.39375229 -99.55918563]
[148.80158552  34.13938441 -98.41245954]
[ 150.35592963   33.52335433 -100.67164534]
[148.83333273  33.70914349 -99.09780031]
[148.28252562  33.51749305 -98.9320604 ]
[147.93071078  33.278819   -99.00277889]
[147.66190694  33.12022297 -99.0200627 ]
[ 149.37918213   32.81736653 -100.9287411 ]
[ 149.25121812   32.77820388 -100.87965549]
[ 149.13863119   32.76024099 -100.8111663 ]
[147.7210414   33.07749313 -99.1338285 ]
[147.80723395  32.63286514 -99.87934757]
[147.14776792  33.03812348 -98.71877718]
[ 148.56289793   32.41727766 -100.84592219]
[ 148.9205085    32.92067316 -100.37944793]
[148.32326338  33.18597398 -99.47103474]
[ 149.50821097   33.48020174 -100.01681704]
[ 149.22734257   32.90962021 -100.65717174]
[148.58305971  33.36099349 -99.42132301]
[ 149.93938409   33.42931953 -100.46173559]
[148.32461357  33.04048353 -99.69385001]
[148.07762404  33.41553464 -98.91708665]
[148.50764035  33.45808436 -99.21003113]
[ 148.9506081    33.01912683 -

...

[146.88291303  33.21864217 -98.22987672]
[146.25777987  33.01110925 -98.02982327]
[146.58214945  33.08483486 -98.1843743 ]
[145.54616817  32.55944656 -98.12143855]
[146.37270052  32.11312522 -99.46438059]
[145.41632296  33.04449368 -97.2990899 ]
[146.09684354  32.33935933 -98.89892839]
[145.5069751   32.46709203 -98.22620969]
[144.89085476  32.32895917 -97.92948358]
[144.28936345  31.91877668 -98.04366964]
[145.99303674  31.95314797 -99.38808234]
[146.69750905  32.09538103 -99.76118886]
[145.19047675  31.96876075 -98.70402663]
[145.61022886  32.51055387 -98.24600693]
[146.35439279  32.49787685 -98.87469326]
[145.47440973  32.2241999  -98.55901266]
[145.16718198  32.08272156 -98.51685792]
[144.54470764  32.25263075 -97.76160328]
[142.89040151  32.29279302 -96.3831666 ]
[146.08266951  32.34891441 -98.87303263]
[ 147.05036559   31.99244997 -100.21103892]
[147.11062117  32.37316555 -99.68870292]
[146.43925225  32.70915841 -98.62894781]
[146.81827829  33.4587414  -97.81695057]
[145.12942606

[147.38662648  32.33167313 -99.98212284]
[145.92197854  32.4375351  -98.60920327]
[ 148.02946031   32.23240992 -100.67437543]
[146.44578224  32.30006962 -99.24565504]
[146.86530047  32.31666742 -99.56911875]
[147.05272973  32.50106042 -99.44806156]
[ 147.73781848   32.39536157 -100.18086332]
[146.97924068  32.34638183 -99.61940556]
[ 147.91601302   32.1691227  -100.67431447]
[145.89593307  31.83912906 -99.47720102]
[146.53087514  32.40166933 -99.16417421]
[146.79232731  32.09489138 -99.84106123]
[ 147.40820417   32.29393922 -100.05717757]
[146.84430923  32.05631464 -99.94239136]
[ 147.64005172   31.86223666 -100.90530488]
[ 146.9359892    31.97810371 -100.13661931]
[146.64726612  31.93724302 -99.95617318]
[146.16881656  31.95357486 -99.5331988 ]
[145.66627543  31.52682876 -99.75034468]
[ 145.65099483   31.21704675 -100.1971346 ]
[ 146.99535634   31.85012927 -100.37881167]
[ 146.42079738   31.66965077 -100.16709674]
[146.16911205  31.76844416 -99.80941558]
[146.51494314  32.73841166 -98

[147.05061203  33.18178048 -98.42273593]
[147.45454621  33.00963718 -99.01531222]
[146.72406114  32.82213507 -98.69409089]
[147.5384867   32.6412594  -99.64166332]
[146.37625921  32.35016183 -99.11330656]
[145.61863857  32.5469829  -98.19889714]
[146.51593597  32.22191343 -99.4206689 ]
[145.35216413  32.6049912  -97.89652484]
[145.46701503  32.14031803 -98.67706454]
[145.46164408  32.69209701 -97.85654621]
[147.3056632   32.49182636 -99.67278356]
[147.84159877  32.80849323 -99.6419027 ]
[146.69590551  32.79607695 -98.70995932]
[147.08120212  32.16586053 -99.97603635]
[147.56791629  32.5516532  -99.80173132]
[ 148.50487211   32.0196408  -101.40348061]
[ 148.74950404   32.39646567 -101.03683266]
[ 149.47398376   32.52144866 -101.4660259 ]
[ 149.05083507   32.41140095 -101.27176201]
[ 148.37613451   32.59214046 -100.42057683]
[ 148.07548632   32.69750379 -100.00655392]
[ 149.74745731   32.47865992 -101.7680533 ]
[ 148.91044183   31.82099874 -102.05629348]
[ 148.87940382   32.39221528 -101

[ 147.42386515   32.25168382 -100.13411888]
[147.10333729  32.19166179 -99.95574975]
[ 147.5622694    32.09837949 -100.48229639]
[ 148.95327656   32.16177902 -101.57090995]
[ 148.23654446   32.62715464 -100.24925759]
[ 148.00911713   32.57885656 -100.13096031]
[146.63366444  32.51126758 -99.08517257]
[146.65747231  32.61145178 -98.954819  ]
[ 148.08551864   32.28451984 -100.64270929]
[147.20676243  32.43893053 -99.66993203]
[146.39423553  32.64157934 -98.69290251]
[147.24119368  32.70469305 -99.29822783]
[145.5346145   32.8359872  -97.70280855]
[146.91070076  33.33536402 -98.07741734]
[147.32116657  32.91148829 -99.05288463]
[146.00779962  33.10572506 -97.68586319]
[146.80338619  32.57335803 -99.13254508]
[145.34240246  32.27427473 -98.37713282]
[145.74138421  32.33943078 -98.60686778]
[146.02689989  32.12799777 -99.1558998 ]
[146.86662729  32.64461189 -99.07797843]
[145.83275084  32.53794398 -98.38700874]
[146.02929018  32.46835131 -98.65139501]
[146.60220389  31.96850365 -99.87171103

[145.72050827  31.77608062 -99.42553334]
[146.33776339  32.56164743 -98.76580522]
[147.29797742  32.7935731  -99.21143569]
[146.23030866  32.9727553  -98.06461535]
[147.15510683  32.93768621 -98.8760086 ]
[ 148.86222143   32.80662124 -100.50479319]
[145.95736324  32.38929284 -98.70994903]
[147.22818585  32.49699762 -99.60033041]
[ 148.3590072    32.73707147 -100.18517674]
[146.35134665  32.24889749 -99.24392105]
[146.10561901  32.00932972 -99.39770419]
[ 147.36098041   31.89685769 -100.61668303]
[ 147.31197284   32.1548898  -100.18615447]
[145.24298491  31.92345022 -98.81393325]
[ 147.92639456   31.96882144 -100.98706545]
[146.01457631  31.77253272 -99.67473532]
[146.03859427  31.97700953 -99.39030392]
[ 147.01440669   31.9650439  -100.22205618]
[146.80405661  32.19090352 -99.70681658]
[145.93903803  32.45553604 -98.59643219]
[ 147.53289095   32.11324073 -100.43503911]
[144.73569057  31.72527747 -98.69034368]
[146.17790044  31.95081808 -99.54485012]
[ 146.91751364   32.01434312 -100.06